<a href="https://colab.research.google.com/github/MariaLanderos/Curso-Python-2023/blob/main/Copia_de_Ch23_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trees
<a href="https://colab.research.google.com/github/rambasnet/FDSPython-Notebooks/blob/master/Ch25-Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://openbookproject.net/thinkcs/python/english3e/trees.html
- like linked lists, trees are made up of nodes

## Binary Tree
 - is a commonly used tree in which each node contains a reference to atmost two other nodes (possibly None)
- these references are referred to as the left and right subtrees
- like the node of linked list, each node also contains data/cargo
- like linked lists, trees are recursive data structures that are defined recursively:
    1. the empty tree, represented by None, or
    2. a node that contains a data and two tree references (left and right subtree)

## Building trees
- similar to building linked-list

In [2]:
class Tree:
    def __init__(self, data, left=None, right=None):
        self.cargo = data
        self.left = left
        self.right = right

    def __str__(self):
        return "{}".format(self.cargo)


### bottom-up way to build-trees
- first create children and link them to the parent

In [3]:
left = Tree(2)
right = Tree(3)
tree = Tree(1, left, right)

In [4]:
tree1 = Tree(10, Tree(20), Tree(30))

In [5]:
print(tree)

1


In [6]:
print(tree1)

10


## traversing tree
- natural way to traverse a tree is recursively!

In [7]:
def findSum(tree):
    if not tree:
        return 0
    return tree.cargo + findSum(tree.left) + findSum(tree.right)


In [8]:
findSum(tree)

6

In [9]:
findSum(tree1)

60

## Árboles de expresión
- Los árboles son una forma natural de representar la estructura de una expresión sin ambigüedades.
- la expresión infija 1 + 2 * 3 es ambigua a menos que sepamos el orden de operación que * ocurre antes de +
- podemos usar árbol para representar la misma expresión
     - los operandos son nodos hoja
     - los nodos operadores contienen referencias a sus operandos; los operadores son binarios (dos operandos)
    
<img src="./resources/tree2.png" />

- aplicaciones:
     - traducir expresiones a sufijo, prefijo e infijo
     - los compiladores usan árboles de expresión para analizar, optimizar y traducir programas
    
- tres formas de atravesar árboles: pre-orden, en orden y post-orden

In [10]:
expression = Tree('+', Tree(1), Tree('*', Tree(2), Tree(3)))

### pre-order tree traversal
- contents of the root appear before the contents of the children
- recursive algorithm:
    - visit the node
    - visit left subtree
    - visit right subtree

In [11]:
def preorder(tree):
    if not tree:
        return
    print(tree.cargo, end=' ')
    preorder(tree.left)
    preorder(tree.right)


In [12]:
preorder(expression)

+ 1 * 2 3 

### in-order tree traversal
- contents of the tree appear in order
- recursive algorithm:
    - visit left subtree
    - visit node
    - visit right subtree

In [13]:
def inorder(tree):
    if not tree:
        return
    inorder(tree.left)
    print(tree.cargo, end=' ')
    inorder(tree.right)

In [14]:
inorder(expression)

1 + 2 * 3 

In [15]:
def inorderIndented(tree, level=0):
    if not tree:
        return
    inorderIndented(tree.right, level+1)
    print('   '*level + str(tree.cargo))
    inorderIndented(tree.left, level+1)

In [16]:
inorderIndented(expression)

      3
   *
      2
+
   1


### post-order traversal
- recursive algorithm:
    1. visit left subtree
    2. visit right subtree
    3. visit node

In [17]:
def postorder(tree):
    if not tree:
        return
    postorder(tree.left)
    postorder(tree.right)
    print(tree.cargo, end=' ')

In [18]:
postorder(expression)

1 2 3 * + 

## building an expression tree
- parse an infix expression and build the corresponding expression tree
- e.g., (3 + 7) * 9 yields the following tree:
<img src="./resources/tree3.png">

1. tokenize expression into python list? How (left as an exercise)
 - (3 + 7) * 9 = ["(", 3, "+", 7, ")", "*", 9, "end"]
2. "end" token is useful for preventing the parser from reading pas the end of the list

In [19]:
def get_token(token_list, expected):
    if token_list[0] == expected:
        del token_list[0]
        return True
    return False

In [20]:
# handles operands
def get_number(token_list):
    x = token_list[0]
    if not isinstance(x, int):
        return None
    del token_list[0]
    return Tree(x, None, None) # leaf node

In [21]:
token_list = [9, 11, 'end']
x = get_number(token_list)
postorder(x)

9 

In [22]:
print(token_list)

[11, 'end']


In [23]:
def get_product(token_list):
    a = get_number(token_list)
    if get_token(token_list, '*'):
        b = get_number(token_list)
        return Tree('*', a, b)
    return a

In [24]:
token_list = [9, '*', 11, 'end']
tree = get_product(token_list)

In [25]:
postorder(tree)

9 11 * 

In [26]:
token_list = [9, '+', 11, 'end']
tree = get_product(token_list)
postorder(tree)

9 

In [27]:
# adapt the function for compound product such as 3 * (5 * (7 * 9))
def get_product(token_list):
    a = get_number(token_list)
    if get_token(token_list, '*'):
        b = get_product(token_list)
        return Tree('*', a, b)
    return a

In [28]:
token_list = [2, "*", 3, "*", 5 , "*", 7, "end"]
tree = get_product(token_list)
postorder(tree)

2 3 5 7 * * * 

In [29]:
#  a sum can be a tree with + at the root, a product on the left, and a sum on the right.
# Or, a sum can be just a product.
def get_sum(token_list):
    a = get_product(token_list)
    if get_token(token_list, "+"):
        b = get_sum(token_list)
        return Tree("+", a, b)
    return a

In [30]:
token_list = [9, "*", 11, "+", 5, "*", 7, "end"]
tree = get_sum(token_list)

In [31]:
postorder(tree)

9 11 * 5 7 * + 

In [32]:
# handle parenthesis
def get_number(token_list):
    if get_token(token_list, "("):
        x = get_sum(token_list)         # Get the subexpression
        get_token(token_list, ")")      # Remove the closing parenthesis
        return x
    else:
        x = token_list[0]
        if not isinstance(x, int):
            return None
        del token_list[0]
        return Tree(x, None, None)

In [33]:
# 9 * (11 + 5) * 7
token_list = [9, "*", "(", 11, "+", 5, ")", "*", 7, "end"]
tree = get_sum(token_list)
postorder(tree)

9 11 5 + 7 * * 

## handling errors on malformed expressions

In [34]:
# handle parenthesis
def get_number(token_list):
    if get_token(token_list, "("):
        x = get_sum(token_list)         # Get the subexpression
        if not get_token(token_list, ")"): # Remove the closing parenthesis
            raise ValueError('Missing close parenthesis!')
        return x
    else:
        x = token_list[0]
        if not isinstance(x, int):
            return None
        del token_list[0]
        return Tree(x, None, None)

In [35]:
token_list = [9, "*", "(", 11, "+", 5, ")", "*", 7, "end"]
tree = get_sum(token_list)
postorder(tree)

9 11 5 + 7 * * 

## Ejercicios propuestos:

1. Modifique la función inorder para que ponga paréntesis alrededor de cada operador y par de operandos. ¿La salida es correcta y sin ambigüedades? ¿Son siempre necesarios los paréntesis?



2. Escriba una función que tome una cadena de expresión y devuelva una lista de tokens.

3. Encuentre otros lugares en las funciones del árbol de expresión donde pueden ocurrir errores y agregue declaraciones de elevación apropiadas. Pruebe su código con expresiones mal formadas.